# core

> a reverse_tb jupyer magic

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from contextlib import contextmanager
from typing import Optional

from fastcore.all import *
from IPython.core.magic import register_cell_magic
from IPython.core import ultratb

In [ ]:
#| export
@contextmanager
def redirect_jupyter_stderr():
    ip = get_ipython()
    showtraceback = ip.showtraceback
    
    try:
        ip.showtraceback = noop
        yield 
    except Exception as e:
        pass
    finally:
        ip.showtraceback = showtraceback


In [ ]:
#| export
class ReverseTB(ultratb.VerboseTB):
    def text(self, etype, value, tb, tb_offset: Optional[int] = None, context=5):
        """Return formatted traceback."""
        tb_list = self.structured_traceback(etype, value, tb,
                                            tb_offset, context)
        tb_list.reverse()
        tb_list = [x.replace('most recent call last', 'last call first') for x in tb_list if '[... skipping hidden 1 frame]' not in x]
        tb_list = tb_list[-1:]+tb_list[:-1]
        return self.stb2text(tb_list)

In [ ]:
#| export
@register_cell_magic
def reverse_tb(line, cell):
    """A cell magic that reverses only the frames of the traceback of an error."""
    # Execute the cell and get the result object
    with redirect_jupyter_stderr():
        result = get_ipython().run_cell(cell)
    # If the execution was not successful, get the error
    if not result.success:
        e = result.error_in_exec
        etype = result.error_before_exec or result.error_in_exec
        tb = result.error_in_exec.__traceback__
        # Get the formatted traceback
        vtb = ReverseTB(color_scheme='Neutral')
        tb_text = vtb.text(type(etype), e, tb)
        print(tb_text)

In [ ]:
def foo():
    return bar()

def bar():
    return baz()

def baz():
    try:
        qux()
    except KeyError as e:
        raise Exception
    return qux()

def qux():
    d = {}
    return d['key']

In [ ]:
foo()

Exception: 

In [ ]:
%%reverse_tb
foo()

---------------------------------------------------------------------------
Exception: 
Cell In[28], line 11, in baz()
      9     qux()
     10 except KeyError as e:
---> 11     raise Exception
     12 return qux()

Cell In[28], line 5, in bar()
      4 def bar():
----> 5     return baz()

Cell In[28], line 2, in foo()
      1 def foo():
----> 2     return bar()

Cell In[30], line 1
----> 1 foo()

Exception                                 Traceback (last call first)

During handling of the above exception, another exception occurred:

KeyError: 'key'
Cell In[28], line 16, in qux()
     15 d = {}
---> 16 return d['key']
        d = {}

Cell In[28], line 9, in baz()
      8 try:
----> 9     qux()
     10 except KeyError as e:

KeyError                                  Traceback (last call first)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()